In [1]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

# 키워드 설정
keyword = "천안"

# Selenium WebDriver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")  # 시크릿 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--headless")  # 브라우저 UI 비활성화 (백그라운드 실행)

driver = webdriver.Chrome(options=chrome_options)


# 수집할 호텔 URL들을 저장할 리스트
hotel_urls = []

for url in agoda_hotel_Seoul:
    # 페이지로 이동
    driver.get(url)
    time.sleep(20)  # 페이지 로드 대기
    
    original_window = driver.current_window_handle
    
    # 1부터 10까지의 호텔을 클릭하고 URL을 가져오기
    for i in tqdm(range(1, 11), desc="Collecting hotel URLs (first 10)"):
        try:
            hotel_xpath = f'//*[@id="contentContainer"]/div[4]/ol/li[{i}]/div'
            hotel_element = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, hotel_xpath))
            )
            driver.execute_script("arguments[0].scrollIntoView();", hotel_element)
            time.sleep(2)
            hotel_element.click()
            time.sleep(20)
            WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))
            driver.switch_to.window(driver.window_handles[-1])
            hotel_url = driver.current_url
            hotel_urls.append(hotel_url)
            driver.close()
            driver.switch_to.window(original_window)
        except Exception as e:
            print(f"Failed to get URL for hotel {i}: {e}")
            break
    
    # 11부터 88까지의 호텔을 클릭하고 URL을 가져오기
    for i in tqdm(range(11, 89)):
        try:
            hotel_xpath = f'//*[@id="contentContainer"]/div[5]/ol/li[{i}]/div'
            hotel_element = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, hotel_xpath))
            )
            driver.execute_script("arguments[0].scrollIntoView();", hotel_element)
            time.sleep(2)
            hotel_element.click()
            time.sleep(20)
            WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))
            driver.switch_to.window(driver.window_handles[-1])
            hotel_url = driver.current_url
            hotel_urls.append(hotel_url)
            driver.close()
            driver.switch_to.window(original_window)
        except Exception as e:
            print(f"Failed to get URL for hotel {i}: {e}")
            break

# 수집한 URL을 DataFrame으로 저장
df_urls = pd.DataFrame(hotel_urls, columns=['Hotel URL'])

# DataFrame을 CSV 파일로 저장
df_urls.to_csv("hotel_urls.csv", index=False)

# 드라이버 종료
driver.quit()



KeyboardInterrupt: 

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

# 함수: 호텔 리뷰 데이터를 추출
def extract_hotel_data(driver, hotel_url):
    hotel_data_list = []

    # 호텔 이름 추출
    try:
        hotel_name_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'h2[data-selenium="hotel-header-name"]'))
        )
        hotel_name = hotel_name_element.text
        print(f"Hotel name: {hotel_name}")
    except Exception as e:
        hotel_name = "Unknown"
        print(f"Failed to get hotel name: {e}")

    # 리뷰 점수 낮은 순으로 정렬
    try:
        sort_option = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="review-sort-id"]/option[3]'))
        )
        sort_option.click()
        time.sleep(5)  # 정렬 적용을 위한 대기 시간
        print("Reviews sorted by lowest score first.")
    except Exception as e:
        print(f"Failed to sort reviews by lowest score: {e}")
        return hotel_data_list  # 정렬에 실패하면 리뷰 추출을 시도하지 않음

    # 리뷰 추출 (최대 50개: 5개씩 10번 반복)
    for page in range(10):  # 10번 반복
        try:
            reviews = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p[data-selenium="comment"]'))
            )
            if not reviews:
                print("No reviews found on this page.")
                break  # 더 이상 리뷰가 없으면 루프 종료
            for i, review in enumerate(reviews[:5]):  # 최대 5개의 리뷰만 가져오기
                review_text = review.text.strip()

                # 후기 점수 추출
                try:
                    review_score_xpath = f'//*[@id="review-{i}"]/div[1]/div/div[1]/div[1]'
                    review_score_element = driver.find_element(By.XPATH, review_score_xpath)
                    review_score = review_score_element.text.strip()
                except Exception as e:
                    review_score = "N/A"
                    print(f"Failed to get review score for review {i}: {e}")

                # 호텔명, URL, 리뷰, 후기 점수를 각각 한 행으로 추가
                hotel_data_list.append({
                    '사이트': "아고다",
                    '호텔 URL': hotel_url,
                    '호텔 리뷰': review_text,
                    '호텔 리뷰 점수': review_score
                })
                print(f"Review: {review_text}, Score: {review_score}")

            # 다음 페이지 버튼 클릭
            try:
                    # 페이지 로드 후 첫 번째 리뷰를 클릭
                    next_button = WebDriverWait(driver, 20).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-panel-1 > div:nth-child(2) > div > nav > butt안}.csv', index=False, encoding='utf-8-sig')


Hotel name: 센텀 빅토리아 호텔 (Busan Centum Victoria Hotel )
Reviews sorted by lowest score first.
Review: ㅋㅋㅋ고장난것도많고 재방문 절대안함 더러움, Score: 2.0
Review: 세면대 머리카락과 욕실세면대 물셈 시설들이대체로 관리가안된것 같았다 쇼파도 군데군데 헤져있었고 모든 시설들이 다시오고 싶지않은 상태였다, Score: 2.0
Review: 저렴한 숙소 자주가고 크게숙소에돈쓰는타입아닌데도 여기는 돈 더 주고라도 다른데 가고싶을정도로 냄새가 너무 별로인데다가 체크인카운터없어 핸드폰 충전기 빌려주지도않아 재고소진이라하는데 애초에 충전기가 없는듯 요즘모텔에도 충전기는 다 있고 카운터 연락이안됨, Score: 2.0
Review: 비대면체크인이라해서 전화했는데 연락이 잘 안됨. 특히 여자분 전화 일부러 죽어라 안받는듯 일행이랑 진짜 너무 화나서 계속 전화했는데 첨에 한번 받고는 절대 안받음. 겨우 연락돼서 방에 들어가니 사람은 없었지만 짐이있었음. 여기서 2차당황+화남 다행히 옆호텔로 오라해서 그쪽으로 옮겨갔고 일부 환불도 해줌. 그런데 우린 습하고 덥고 짐도 들고있는상태에서 1시간 넘게 기다림. 그리고 센텀호텔 13만원정도 되는 돈주고 묵고싶은곳은 아니였음 터가 이상한지 멀쩡한 폰도 여기선 잘 안터짐 공짜로 줘도 다신 안가고싶음. 여행 첫날부터 시간날리 고생하고 기분 상하는 곳임., Score: 2.0
Review: 방은 넓었습니다. 다만 청소는 따로 별도 비용을 내야하는 것이 단점., Score: 2.0
Failed to click next page button: Message: element click intercepted: Element <button aria-label="다음 이용후기 페이지" leadingicon="fill.arrow.thin-forward" data-element-name="review-paginator-next" type="button" class="a

Hotel name: 어반스테이 부산송도해변 (Urbanstay Busan songdo Beach)
Reviews sorted by lowest score first.
Review: 체크인 개판 침대 모래굴러다님, Score: 2.0
Review: 서비스가 꽝이네요, Score: 2.0
Review: 인생최악의호텔, Score: 2.0
Review: 건물내주차 불가 차량의 경우 호텔 근처 주차 어려움. 호텔에서도 대기외에는 대책없음. 한시간이상대기., Score: 3.2
Review: 그랩디오션에서 하루 묵어서 똑같은 건물이라 좋을거라 생각하고 예약했는데 완전 다름 체크인을 키오스크에서 셀프로 해야 하는데 비밀번호가 문자로 오지도 않음 전화하니 받지도 않고 어렵게 통화됐는데 내 뒤에 나처럼 열받은 사람들 넘쳐났음 침대 매트리스도 라꾸라꾸 수준이고 옆사람이 움직이면 나도 출렁임 청소도 그랩보다 별로고 수건이 회색인데 냄새가 장마때 나는 냄새남 바닷가 바로 앞인 위치가 좋은거 빼고는 그랩디오션과는 수준차이 느껴졌음, Score: 3.6
Failed to click next page button: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#reviews-panel-1 > div:nth-child(2) > div > nav > button:nth-child(3)"}
  (Session info: chrome=128.0.6613.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C811B632+29090]
	(No symbol) [0x00007FF7C808E6E9]

Hotel name: 센텀 프리미어 호텔 (Centum Premier Hotel)
Reviews sorted by lowest score first.
Review: 우선 방에 입실시 퀘퀘한 냄새가 났고 매우좁았고 새벽에 밖에서 피우는 담배 냄새가 방으로 들어와 힘들고 원래 조식포함 하루 5만6천원으로 아고다에서 예약했는데 청구한비용은 7만6천원으로 결제해깄고 프론트에 문의 했으나 아고다에게 물어보라고함 자기네는 아고다측에서 청구란데로 청구했다고함, Score: 2.0
Review: 체크인 후 입실하자마자 세면대 머리카락, 카펫 바닥에 소톱 같은 것. 둘째날 일 마치고 돌아와 메이킹한 침대 누우려고 이불 들추니 치대커버에 벌레 시체. 엘리베이터 매우 느림. 청소수레가 고객용 엘리베이터로 다님, Score: 3.6
Review: 입실할때부터 약품냄새인지 룸에서 약냄새가 진동해 밤새 머리가 아파 잠도 못잤고 거머가 천장에서 줄타고 내려옵니다 프런트도 없구요 직원이 만나서 안내해주시고 직원분 친절한거 빼고는 호텔맞나요?, Score: 3.6
Review: 전기차 충전 시설에 대한 안내도 없어 아쉽습니다., Score: 4.0
Review: 무슨 호텔이 와인오프너도 구비가 안되어 있습니끼? 호텔 매니저는 그거 하나 준비 안해둬서 투숙객한테 매일같이 시달릴 호텔 직원들 생각도 못하는 사람인가봅니다. 객실내 가운은 다 찢겨서 입지도 못했습니다. 센텀 프리미어랑 같은 호텔을 사용하는지 여기저기 센텀 프리미어의 흔적이 있더군요. 엘레베이터는 22층 호텔에 한개밖에 없어서 한반 내려가려면 얼마나 오래 기다리는지 참... 에휴 여기 간 내가 병신이지 이돈주고 다시는 안갑니다., Score: 4.0
Failed to click next page button: Message: element click intercepted: Element <button aria-label="다음 이용후기 페이지" leadingicon="fill.arrow.thin-forward" data-elemen

Failed to get hotel name: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C811B632+29090]
	(No symbol) [0x00007FF7C808E6E9]
	(No symbol) [0x00007FF7C7F4B1CA]
	(No symbol) [0x00007FF7C7F9EFD7]
	(No symbol) [0x00007FF7C7F9F22C]
	(No symbol) [0x00007FF7C7FE97F7]
	(No symbol) [0x00007FF7C7FC672F]
	(No symbol) [0x00007FF7C7FE65D9]
	(No symbol) [0x00007FF7C7FC6493]
	(No symbol) [0x00007FF7C7F909B1]
	(No symbol) [0x00007FF7C7F91B11]
	GetHandleVerifier [0x00007FF7C843883D+3294125]
	GetHandleVerifier [0x00007FF7C8484423+3604371]
	GetHandleVerifier [0x00007FF7C847A2E7+3563095]
	GetHandleVerifier [0x00007FF7C81D6F16+797318]
	(No symbol) [0x00007FF7C809986F]
	(No symbol) [0x00007FF7C8095454]
	(No symbol) [0x00007FF7C80955E0]
	(No symbol) [0x00007FF7C8084A7F]
	BaseThreadInitThunk [0x00007FFB57E0257D+29]
	RtlUserThreadStart [0x00007FFB58E6AF28+40]

Reviews sorted by lowest score first.
Review: 주차비용지불해야되요 주차타워 지하3층까지 차량 승강기 엄청비좁고 위험합니다. 여자혼자 탑승하기 불안합니다., Score: 2.0
Review: ㄱ, Score: 2.0
Review: 엘

Hotel name: 어반스테이 서면 (Urbanstay Seomyeon)
Reviews sorted by lowest score first.
Review: 체크인 시간도 지연되고 세면대, 침구류 청소, 위생상태 별로여서 불만족합니다, Score: 2.0
Review: 가격대비 좋은점 1도 찾을수 없었습니다., Score: 2.0
Review: 돈주고 숙소를 이용하는데 방에들어갓을때 청소안된 침대와 블라인드엔 토를 한거 마냥 오염된채 있었습니다 다음날 문의할려고 하기전에 전화와서는 저희보고 블라인드 오염햇냐고 물으시는 직원의 말에 너무 황당하고 놀랫죠 .. 그럼 저희가 사용하기 전까지 블라인드가 오염된걸 몰랐던거고 청소을 안했다는 것에 너무 불쾌하고 역겨웟습니다 다시는 이 숙소에 이용하지 않을 예정이고 또다른 피해자가 앖엇으면 하는 바램에 후기 올립니다, Score: 2.0
Review: 투숙객이 무슨 일이 있을 때 대응할 직원 한명 정도는 있어야 하는데, 아무것도 없음. 심지어 화재 경보가 울려도 수습하는 직원 한명 없음. '어반스테이 서면' 말고 가성비 좋은 호텔 부산에 많습니다. 인건비 등을 절감하여 저렴한 서비스 공급한다는 원칙은 알겠으나, 이렇게 관리할거면 호텔이라는 타이틀은 기재하지 마세요. 요새 게스트 하우스도 이렇게 불청결하게 관리 하지 않습니다. 어중이 떠중이 다 들락거리는 대학교 앞 친구 자취방도 아니고, 남이 지내던 오피스텔 레지던스에, 청소도 안된 상태에서 그대로 들어가서 지내다 나오시는 거라고 생각하시면 됩니다., Score: 2.4
Review: 투숙객 주차비15000원 부과 출차 후 오후9시 이후 주차 할 수없음. (주차장소 협소하여 주차관리요원 퇴근 이후 주차 불가), Score: 2.8
Failed to click next page button: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#reviews-panel-

...

Hotel name: 와이컬렉션 by UH FLAT 오시리아 (Y Collection by UH FLAT Osiria)
Reviews sorted by lowest score first.
Review: 11시00퇴실. 시간인데. 10시00. 청소하시는데 좀. 기분이. 좋지는 않았어요. 그리고. 과일 먹는데. 포크 없고 숙소. 도착했는데 사진상에 나오는뷰도. 아닌고 귀찮아 그냥 2박3일. 지내서 나왔어요, Score: 2.0
Review: 건물전체에서 비린내와 세제냄새가 섞여 진동하고 침대 스프링은 앉을수도 없는정도임. 가방놓고 앉았다 일어서니 가방안 물건이 다튕겨 쏟아지는정도..ㅠ 매트요청하니 5만원 이라고.. 화장실냄새와 수압은 애교.. 거기다 층간소음에 어나더레벨을 경험하고옴..방바꿔도 소용없었음..거기 사는사람도있던데 정말 대단들하십니다., Score: 2.0
Review: 추가 유료이불(1일 1개 2만원) 이틀 지불하였는데 갖다놓은건 이불대신 베개2개ㅋㅋ객실도 청소상태 안좋음 머리카락 투성이. 3박묵었는데 마지막 날은 청소는 커녕 수건 교체도 없고 물도 추가해주지 않음., Score: 2.0
Review: 별로, Score: 2.0
Review: 대가족 여행으로 방을 2개 예약했는데 체크인 직전에 방 2개 중 한곳에 문제가 생겼다며 다른 방으로 안내를 해줌 뷰 때문에 고층으로 예약을 했었는데 바뀐 방은 뷰도 별로인 저층으로 해주심 체크인 후 화장실에 들어갔는데 청소가 덜 되어 있었음 이전 사람이 쓴 건지 뭔지 모를 때수건이 널브러져 있었고 또 점검을 제대로 안하셨는지 트리트먼트가 고장나 있었음 밤에 발견해서 다음날 말하니 그제서야 고쳐주심 총 2박을 예약했어서 중간에 청소를 해주신다길래 어른들이 쓰는 곳은 전체적인 청소는 해주실 필요 없고 쓰레기 정리와 기본적인 물품 리필만 부탁했는데 언급도 하지 않은 아이들이 쓰는 방은 청소도 해주시지 않고 기본적으로 주시는 물과 수건도 리필되어있지 않았음 아이들 방에 물이랑 수건을 요구하자 귀찮다는 듯이 응대를 하시

Hotel name: 그리핀 베이 호텔 (GRIFFIN BAY HOTEL)
Reviews sorted by lowest score first.
Review: 최악, Score: 2.0
Review: 담배냄새나고 욕조도없고 벽걸이TV균형도 안맞고 카드키는 인식이 계속 안되서 몇번을 바꾸러가고 수리직원이 오시고 그런데도 조식 이용하고 또 인식안되서 결국에는 청소해주시는 여사님이 열어주심 대실하고 방 내준듯 방금쓴 느낌나고.... 아이데리고는 절대 이용안할뿐더러 앞으로 개인여행으로도 이용계획없음 조식도 맛있다고 유명하다던데 전부 인스턴트...., Score: 2.0
Review: 아고다로 여기 예약하고 완전 여행 망침 절대 비주천, Score: 2.0
Review: 도착하자마자 누런 귓밥이 그대로 묻어있는 면봉이 화장대 위에 남겨져있었고 직원분께 말씀드리니 사람이 하는 일이다보니 그럴 수 있다셨습니다. 마침 관리자 같은 분이 조정해주셨는지 나중엔 한박만 묵고 다른 날짜는 환불해주겠다해서 겨우 벗어났지만 난방이 전혀 안 돼서 너무너무 추웠고 결국 한 숨도 못 자고 나왔어요 조식은 강추에요, Score: 2.0
Review: 방 들어갔는데 퀘퀘한 냄새가 나서 20분 고민하다 변경 요청했는데 와서 방향제 뿌려주고 가시네요. 티비도 아쉽고요, Score: 2.4
Failed to click next page button: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#reviews-panel-1 > div:nth-child(2) > div > nav > button:nth-child(3)"}
  (Session info: chrome=128.0.6613.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/

Hotel name: MS 호텔 해운대 (MS Hotel Haeundae)
Reviews sorted by lowest score first.
Review: ss, Score: 2.0
Review: 아침 8시30분부터 차빼고 서울가야해서 내려갓는데 주차타워고장나서 10시반정도까지출발못하고 밖에서기다람 어떠한보상도없음 건물측과 호텔측 서로 남탓하기바쁨 시간날림, Score: 2.0
Review: 위치는 좋습니다만 시설이 조금씩 부실하고 방크기가 작습니다, Score: 2.0
Review: 8월 20일에 예약했는데 집회로 인해 코로나가 다시 반발하여 여행을 못가게 되었습니다. 그래서 호텔 측에서 기간을 미룰 수 있게 도와주웠고 저희는 2주뒤로 예약을 미뤘습니다. 하지만 코로나는 더욱 심해졌고, 사회적 거리두기도 더욱 강화되어 수도권에 사는 저로선 여행을 갈 수 없는 상황이 되어 환불이나 기간을 더 늘려 줄 수 없는지 문의을 드렸습니다. 근데 호텔 측 에서 여행 자제 권고 사항이지 강제가 아니여서 여행을 올 수 있다고 판단이 되었다고 환불은 안되고, 기간도 이미 늘려주워서 안된다고 하더군요..이런 상황에 여행을 가라는게 말이 됩니까...?그냥 돈을 날리라는 소리로 들렸습니다. 아무리 환불 불가 상품이라고 해도 이러한 어쩔수 없는 특수한 상황에도 적절한 조치를 취해주지 않은 호텔에 크게 실망했습니다., Score: 2.0
Review: 1. 도로가 너무 가까워 배달 오토바이소리 사람들 소음 잠을 전혀 못잠 2. 마스크 안쓴 외국인들 너무 많아 코로나 위험 큼 3. 커피머신 고장, 조식제공 없음. 사전 안내 없음 4. 창문이 더러워 밖이 잘 보이지도 않음 5. 세면기 바닥 물이 세서 세수 할 때마다 물바다 6. 보안 철저해보이지만, 아무나 들어오게 비상 계단 문 열려있음. 누군가 열었지만 제지하지 않음 7. 콘센트 위치도 다 너무 동떨어진 위치에 있음. 세상 불편 8. 이 모든게 다 너무 힘들어서, 3박4일 이지만, 내돈 물고 2박3일만 하고 다른 숙소로 감., Sco

Hotel name: 엘시티 레지던스 (LCT Residence)
Reviews sorted by lowest score first.
Review: 해운대 위치하여 해운대 주변관광을 도보로 다닐수 있어서 좋았습니다., Score: 2.0
Review: 오션스위트에 아이 2 포함 6명 숙박했습니다 기준인원 4인에 2명 추가요금 10만원, 고층객실지정요금 3만원, 밥그릇1개 파손 2만원 추가부담했습니다 오션스위트보다 한단계 더 위 객실은 6인기준이라 추가요금부담할필요 없었는데 오션스위트(4인기준)랑 7~9만원정도 차이가 나더군요 아고다를 비롯해 다른 사이트들 다 둘러봐도 추가요금이 36개월 이상 인당5만원 부과된다는 고지 없습니다 둘째아이 38개월이라도 5만원 내야된다고하더군요 인당 5만원이나 부과될것같으면 당연히 6인기준 선택하죠 돈을 더내고 더 작은객실 이용하게되니 기분이 매우 상하더군요 그것도 예약취소변경이 불가능한 숙박 하루전날 연락와서 추가요금 5만원씩 부과해버리니 안갈수도 변경할수도 없었습니다 그리고 고층객실요청하니 또 돈3만원 더내라고하네요 아이가 높은 층을 원해서 또 돈을 내서 고층 받긴 했는데 좀 심하네 싶은 생각이 들더군요 밥그릇 깨진거야 머 우리 실수니 돈은 내는게 맞지만 밥그릇 하나에 2만원도 적지않고 보증용 카드 체크인할때 줬는데도 직원이 기재를 잘못했다면서 전화를 몇번이나 하는지 닥달하듯이 가기전부터 다녀온 뒤까지 기분이 나쁘네요 가시는 분들은 추가요금 부분 미리 체크하시고 예약하세요 그리고 고층 지정 요금은 안내시는걸 추천합니다 거의다 고층이더라고요 제 느낌일수도 있지만 저층은 거의 못봤습니다 바닥 대리석이라 그릇 떨어지면 바로 깨집니다 조심하세요 여튼 돈만 뜯기고 온 느낌이 계속남아 불쾌하네요, Score: 2.0
Review: 내가 꿈꾸고 살고싶은 아주 깔끔 모던한 곳에서 하루밤 영원이 간직 하고 싶어요 내 머리속에ㅡㅡ, Score: 2.4
Review: 네 식구 모두 100% 이상 만족했어요. 친절, 청결, 위치-뷰도 굿! ^^, 

Failed to get hotel name: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C811B632+29090]
	(No symbol) [0x00007FF7C808E6E9]
	(No symbol) [0x00007FF7C7F4B1CA]
	(No symbol) [0x00007FF7C7F9EFD7]
	(No symbol) [0x00007FF7C7F9F22C]
	(No symbol) [0x00007FF7C7FE97F7]
	(No symbol) [0x00007FF7C7FC672F]
	(No symbol) [0x00007FF7C7FE65D9]
	(No symbol) [0x00007FF7C7FC6493]
	(No symbol) [0x00007FF7C7F909B1]
	(No symbol) [0x00007FF7C7F91B11]
	GetHandleVerifier [0x00007FF7C843883D+3294125]
	GetHandleVerifier [0x00007FF7C8484423+3604371]
	GetHandleVerifier [0x00007FF7C847A2E7+3563095]
	GetHandleVerifier [0x00007FF7C81D6F16+797318]
	(No symbol) [0x00007FF7C809986F]
	(No symbol) [0x00007FF7C8095454]
	(No symbol) [0x00007FF7C80955E0]
	(No symbol) [0x00007FF7C8084A7F]
	BaseThreadInitThunk [0x00007FFB57E0257D+29]
	RtlUserThreadStart [0x00007FFB58E6AF28+40]

Reviews sorted by lowest score first.
Review: 광안리를 좋아해서 조카들과 비싸지만 바닷가 바로 앞 호텔 예약했지만 광안리 끝에 위치한 호텔로 위치 좋지 않고, 차가 없다면 최악의 위치. 나이 많은 여자직원 정말 최악의 불친절..말도

In [ ]:
# 결과를 DataFrame으로 변환 (리뷰 한 개당 한 행)
df_hotel_reviews = pd.DataFrame(all_hotel_reviews)

# 결과 출력
print(df_hotel_reviews)

# 결과를 CSV 파일로 저장
df_hotel_reviews.to_csv('부산_hotel_reviews.csv', index=False, encoding='utf-8-sig')
